# Summer School: Digitale Methoden der Zeitungsanalyse

Dieses Jupyter Notebook zeigt, wie Zeitungstitel (am Beispiel der Zeitung „[La otra Alemania](https://www.deutsche-digitale-bibliothek.de/newspaper/2149754-0)“) aus dem Zeitungsportal über die Programmierschnittstelle (API) der Deutschen Digitale Bibliothek heruntergeladen werden können. Dazu werden Aspekte der API erklärt und der Zugriff mit Python-Programmcode demonstriert.

## Suchindizes der Deutsche Digitale Bibliothek
Die Deutsche Digitale Bibliothek betreibt [Solr](https://solr.apache.org/guide/8_8/searching.html)-Suchindizes, die für die verschiedenen Funktionen der (Sub-) Portale benötigt werden. Das Zeitungsportal benutzt zwei Suchindizes. Eine weiterführende Dokumentation befindet sich hier: https://api.deutsche-digitale-bibliothek.de/#/search/getSolrSearch

- `newspaper`: enthält Informationen über Zeitungstitel
  - Schema: https://dev.fiz-karlsruhe.de/stash/projects/DDB/repos/ddb-backend/browse/Cortex/conf/solr/newspaper/conf/schema.xml
  - Konfiguration: https://dev.fiz-karlsruhe.de/stash/projects/DDB/repos/ddb-backend/browse/Cortex/conf/solr/newspaper/conf/solrconfig.xml
- `newspaper-issues`: enthält die zeitungsbezogenen Metadaten inkl. Volltexte
  - Schema: https://dev.fiz-karlsruhe.de/stash/projects/DDB/repos/ddb-backend/browse/Cortex/conf/solr/newspaper-issues/conf/schema.xml
  - Konfiguration: https://dev.fiz-karlsruhe.de/stash/projects/DDB/repos/ddb-backend/browse/Cortex/conf/solr/newspaper-issues/conf/solrconfig.xml

## Suchindex `newspaper`
Der Suchindex `newspaper` ist ein Suchindex über alle Zeitungstitel der [Zeitschriftendatenbank (ZDB)](https://zdb-katalog.de/). Im Schema des Suchindex (s.o.) sind die Suchfelder (Facetten) dokumentiert. Wenn Zeitungstitel gefunden werden sollen, die im Zeitungsportal verfügbar sind, dann muss nach `hasLoadedIssues:true` (Feld:Wert) gesucht werden.

- https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper/select?q=hasLoadedIssues:true

Die Suchfelder können beliebig miteinander kombiniert werden. Das geht mit den Operatoren `AND` und `OR`. Möchte man beispielsweise Zeitungen mit „La otra Alemania“ im Titel suchen, die auch im Zeitungsportal verfügbar sind, dann kombiniert man: `hasLoadedIssues:true AND title:"La otra Alemania"`

- [https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper/select?q=hasLoadedIssues:true AND title:"La otra Alemania"](https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper/select?q=hasLoadedIssues:true%20AND%20title:"La+otra+Alemania")

### Python-Programmcode

Python-Programmcode in Jupyter Notebooks ermöglicht interaktives Programmieren und sofortige Anzeige von Ergebnissen. Es ist ideal für die Datenanalyse und Visualisierungen. Python-Bibliotheken können in Notebooks nachgenutzt werden und erhöhen so den Funktionsumfang.

Die o. g. Solr-Abfrage kann mit Python ausgeführt werden (Bibliothek [`requests`](https://pypi.org/project/requests/)) und die Antwort von der API mit einem JSON-Parser (`json`) gelesen werden. Eine andere Möglichkeit ist, den Solr-Client [`pysolr`](https://pypi.org/project/pysolr/) zu benutzen. Dieser muss zunächst in der Umgebung mit `pip install -q pysolr` (oder ggf. mit Conda: `conda install conda-forge::pysolr`) installiert werden.

In [1]:
# Python-Bibliothek pysolr installieren
%pip install -q pysolr

Note: you may need to restart the kernel to use updated packages.


### KI-generierter Programmcode

Die Erstellung des Python-Programmcodes ist KI-gestützt möglich. Die folgenden KI-Prompts sind mit ChatGPT (GPT-4o) erfolgreich getestet und liefern (oft 😉) das gewünschte Ergebnis.

<div class="alert alert-block alert-info">
<b>Prompt:</b> Ich möchte in Python mit dem Solr-Client <code>pysolr</code> auf den Endpunkt <code>https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper</code> zugreifen. Kannst Du mir einen Python-Code erstellen, der im Feld <code>location</code> nach „Buenos Aires“ sucht und auch <code>hasLoadedIssues</code> auf „wahr“ setzt. Gibt bitte <code>id</code>, <code>title</code>, <code>location</code>, <code>frequency</code> und <code>progress</code> für jeden Suchtreffer aus.
</div>

In [3]:
import pysolr

# Solr-Endpunkt-URL
solr_url = 'https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper'

# Solr-Client initialisieren
solr = pysolr.Solr(solr_url, timeout=10)

# Suchparameter
q = {
    'q': 'location:"buenos aires" AND hasLoadedIssues:true',
    'fl': 'id,title,location,frequency,progress',
    'rows': 10  # Anzahl der zurückzugebenden Ergebnisse
}

# Suche ausführen
results = solr.search(**q)

# Ergebnisse ausgeben
for result in results:
    print(f"ID: {result.get('id', 'N/A')}")
    print(f"Title: {result.get('title', 'N/A')}")
    print(f"Location: {result.get('location', 'N/A')}")
    print(f"Frequency: {result.get('frequency', 'N/A')}")
    print(f"Progress: {result.get('progress', 'N/A')}")
    print("-" * 40)

ID: 2232340-5
Title: ['Das andere Deutschland']
Location: ['Buenos Aires']
Frequency: ['http://id.loc.gov/vocabulary/frequencies/mon']
Progress: ['[1.]1937/38,Juli-Dez.=Nr. 3-8; 2.1939,Jan. - 5.1941,Dez. = Nr. 9-45; mehr nicht digitalisiert']
----------------------------------------
ID: 2149754-0
Title: ['La otra Alemania']
Location: ['Montevideo', 'Buenos Aires']
Frequency: ['http://id.loc.gov/vocabulary/frequencies/irr']
Progress: ['5.1942,Jan. - 6.1943,15.Dez. = Nr. 46-79; [N.F.] 1.1944,1/2(Febr.)-3/4(März); 7.1944/45,Apr.-15.Dez.=Nr. 80/81-108; 8.1946,1.Jan.-15.Aug.=Nr. 109-124; 1946,1.Sept. - 1949,Jan. = Nr. 125-175; damit Ersch. eingest.']
----------------------------------------


## Suchindex `newspaper-issues`

Der Suchindex `newspaper-issues` ist ein weiterer Suchindex des Zeitungsportals. Dieser enthält alle Ausgaben (`type:issue`) einer Zeitung und alle Seiten (`type:page`). Wenn man nur in einem Zeitungstitel suchen möchte, so kann dies über `zdb_id:{ID der Zeitschriftendatenbank}` (für die „La otra Alemania“ ist es `zdb_id:2149754-0`) eingegrenzt werden.

### KI-generierter Programmcode

<div class="alert alert-block alert-info">
<b>Prompt:</b> Schreibe ein Python-Code, der mithilfe der <code>pysolr</code>-Bibliothek eine Suche in einem Solr-Index durchführt und die Ergebnisse in ein Pandas DataFrame überführt. Der Solr-Index ist über die URL <code>https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper-issues</code> erreichbar. Die Suchabfrage soll nach Dokumenten mit der <code>zdb_id</code> „2149754-0“ und dem <code>type</code> „issue“ suchen und bis zu 1000 Ergebnisse zurückgeben. Anschließend sollen die Ergebnisse in ein Pandas DataFrame überführt und angezeigt werden.
</div>

In [26]:
# Python-Bibliothek pandas installieren
%pip install -q pandas

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd

solr_url = 'https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper-issues'
solr = pysolr.Solr(solr_url, always_commit=True, timeout=10)

q = {
    'q': 'zdb_id:2149754-0 AND type:issue',
    'rows': 1000
}

response = solr.search(**q)
    
# Überführen der Ergebnisse in ein Pandas DataFrame
df = pd.DataFrame(response.docs)

# DataFrame anzeigen
df

,id,paper_title,provider_ddb_id,provider,zdb_id,ns_disclaimer_required,publication_date,place_of_distribution,language,thumbnail
0,ZYOWCJLQOY4U2C5DXWWX22WYDP5PQJT3,La otra Alemania : órgano de los alemanes demo...,UJVQP2TIF4YVCCJQXWZ2BNEECO7ZHYTW,Deutsches Exilarchiv,2149754-0,False,1945-06-15T12:00:00Z,"[Montevideo, Buenos Aires]","[spa, ger]",80cc1929-c89b-4bdf-854d-59b2cab45496
1,MMZANT7CHYIXHM2E63NLTPIQL5VWGIFX,La otra Alemania : órgano de los alemanes demo...,UJVQP2TIF4YVCCJQXWZ2BNEECO7ZHYTW,Deutsches Exilarchiv,2149754-0,False,1944-06-01T12:00:00Z,"[Montevideo, Buenos Aires]","[spa, ger]",fe845673-fd4f-4da4-975d-02c54743b76a
2,GCADKIMUKSSL23LL6BXHWRNP66KQR37R,La otra Alemania : órgano de los alemanes demo...,UJVQP2TIF4YVCCJQXWZ2BNEECO7ZHYTW,Deutsches Exilarchiv,2149754-0,False,1947-04-15T12:00:00Z,"[Montevideo, Buenos Aires]","[spa, ger]",cd7057bb-5d83-449d-a8c5-afa5c1769e8e
3,S3LEVONENQZ7BAICDWHE7L2LPJ6IJYD2,La otra Alemania : órgano de los alemanes demo...,UJVQP2TIF4YVCCJQXWZ2BNEECO7ZHYTW,Deutsches Exilarchiv,2149754-0,False,1944-04-25T12:00:00Z,"[Montevideo, Buenos Aires]","[spa, ger]",9e7862fc-e09a-49c5-999c-b7c5f45ec3b6
4,QC2SMVMRK67PTHDQW65FLNUCUOWUO2UV,La otra Alemania : órgano de los alemanes demo...,UJVQP2TIF4YVCCJQXWZ2BNEECO7ZHYTW,Deutsches Exilarchiv,2149754-0,False,1943-03-01T12:00:00Z,"[Montevideo, Buenos Aires]","[spa, ger]",415e6569-a480-4d25-a2b1-b61202f24f9f
...,...,...,...,...,...,...,...,...,...,...
124,HHJX6JF6XZXGNA22FSHKMDDVP5KATS4J,La otra Alemania : órgano de los alemanes demo...,UJVQP2TIF4YVCCJQXWZ2BNEECO7ZHYTW,Deutsches Exilarchiv,2149754-0,False,1947-07-01T12:00:00Z,"[Montevideo, Buenos Aires]","[spa, ger]",b7c6b3cc-1a27-4410-a0d7-c24f7c732d67
125,SZXP6GEGBOD42J5TKYQ64XNDTUJO2S77,La otra Alemania : órgano de los alemanes demo...,UJVQP2TIF4YVCCJQXWZ2BNEECO7ZHYTW,Deutsches Exilarchiv,2149754-0,False,1945-10-01T12:00:00Z,"[Montevideo, Buenos Aires]","[spa, ger]",9146f0e1-7d4c-473b-9876-056490e778c0
126,CAMRPRGLRHFMUJFGTVKX4RW5KJTRR5LT,La otra Alemania : órgano de los alemanes demo...,UJVQP2TIF4YVCCJQXWZ2BNEECO7ZHYTW,Deutsches Exilarchiv,2149754-0,False,1943-12-15T12:00:00Z,"[Montevideo, Buenos Aires]","[spa, ger]",6f77dacd-03c5-4160-b71d-4280f5493099
127,DGJVAWT6K23ZQ5TVOSAKADSDJSSYQV3V,La otra Alemania : órgano de los alemanes demo...,UJVQP2TIF4YVCCJQXWZ2BNEECO7ZHYTW,Deutsches Exilarchiv,2149754-0,False,1947-10-01T12:00:00Z,"[Montevideo, Buenos Aires]","[spa, ger]",1e6f3bf9-dcb7-42bd-8416-bafd35212352


### Erste Datenanalyse

In dem Dataframe können nun Datenanalysen vorgenommen werden. Wir wollen den Publikationszeitraum von der Zeitung „La otra Alemania“ ermitteln.

In [7]:
# Sicherstellen, dass publication_date als Datumswerte formatiert sind
df['publication_date'] = pd.to_datetime(df['publication_date'])

# Frühestes und spätestes Datum ermitteln
earliest_date = df['publication_date'].min()
latest_date = df['publication_date'].max()

# Ergebnisse anzeigen
print(f"Frühestes Veröffentlichungsdatum: {earliest_date}")
print(f"Spätestes Veröffentlichungsdatum: {latest_date}")

Frühestes Veröffentlichungsdatum: 1942-01-01 12:00:00+00:00
Spätestes Veröffentlichungsdatum: 1949-01-01 12:00:00+00:00


## Download der METS/MODS-Daten

Im Deutschen Zeitungsportal ist jede Zeitungsausgabe durch eine METS/MODS-Datei repräsentiert. Diese beinhaltet Links zu den Bild- und Volltextseiten sowie weitere Informationen.

> METS ([Metadata Encoding and Transmission Standard](https://www.loc.gov/standards/mets/)) und MODS ([Metadata Object Description Schema](https://www.loc.gov/standards/mods/)) sind zwei miteinander verbundene Metadatenformate, die häufig in Bibliotheken und Archiven verwendet werden.
>
> METS ist ein XML-basiertes Format, das zur Kodierung und Übertragung von Metadaten für digitale Bibliotheksobjekte entwickelt wurde. Es dient als Container, der verschiedene Arten von Metadaten und die strukturellen Beziehungen zwischen den Teilen eines digitalen Objekts beschreibt. Es besteht aus mehreren Abschnitten wie `metsHdr` (Header), `fileSec` (Dateien), `structMap` (Struktur) und `metadataSec` (Metadatensektion).
> 
> MODS ist ebenfalls ein XML-basiertes Format, das zur Erfassung und zum Austausch bibliografischer Informationen entwickelt wurde. Es bietet Elemente wie `titleInfo` (Titelinformation), `name` (Namen), `originInfo` (Herkunftsinformationen), und `subject` (Themen).

Auf die METS/MODS-Daten kann über die [API-Methode](https://labs.deutsche-digitale-bibliothek.de/app/ddbapi/#/items/getItemsIdSourceComponent) `items/{id}/source/record` zugegriffen werden. Dafür muss die DDB-ID der Zeitungsausgabe bekannt sein, die wir im vorhergehenden Schritt (bspw. `ZYOWCJLQOY4U2C5DXWWX22WYDP5PQJT3`) ermittelt haben. Damit ergibt sich folgende API-Abfrage:

- https://api.deutsche-digitale-bibliothek.de/2/items/ZYOWCJLQOY4U2C5DXWWX22WYDP5PQJT3/source/record

### KI-generierter Programmcode
<div class="alert alert-block alert-info">
<b>Prompt:</b> Erstelle mit Python ein Verzeichnis <code>La_Otra_Alemania</code>, in dem heruntergeladene XML-Dateien gespeichert werden können. Iteriere durch jede Zeile des bestehenden DataFrames <code>df</code>, der die Spalten <code>id</code> und <code>publication_date</code> enthält.
Für jede Zeile:
<ol>
  <li>Extrahiere den Wert der Spalte <code>id</code>.</li>
  <li>Formatiere den ISO-DateTime-Wert der Spalte <code>publication_date</code> im Format <code>YYYY-MM-DD</code>.</li>
  <li>Generiere eine URL <code>https://api.deutsche-digitale-bibliothek.de/2/items/{id}/source/record</code> zur API-Abfrage</li>
  <li>Setze die HTTP-Header so, dass die Antwort im XML-Format akzeptiert wird.</li>
  <li>Erstelle einen Dateipfad für die XML-Datei im Format <code>{publication_date}_{id}.xml</code> und speichere sie im erstellten Verzeichnis.</li>
</ol>
</div>

In [31]:
# Python-Bibliothek requests installieren
%pip install -q requests

Note: you may need to restart the kernel to use updated packages.


In [9]:
import os
import requests

# Verzeichnis für die XML-Dateien erstellen
directory = 'La_Otra_Alemania'
if not os.path.exists(directory):
    os.makedirs(directory)

# Durch jede Zeile des DataFrames iterieren
for index, row in df.iterrows():
    # Extrahiere den Wert der Spalte id
    item_id = row['id']
    
    # Formatiere den Wert der Spalte publication_date im Format YYYY-MM-DD
    publication_date = row['publication_date'].strftime('%Y-%m-%d')
    
    # Generiere die URL zur API-Abfrage
    url = f'https://api.deutsche-digitale-bibliothek.de/2/items/{item_id}/source/record'
    
    # Setze die HTTP-Header
    headers = {
        'Accept': 'application/xml'
    }
    
    # API-Anfrage senden
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        # Erstelle einen Dateipfad für die XML-Datei
        file_path = os.path.join(directory, f'{publication_date}_{item_id}.xml')
        
        # Speichere die XML-Datei im erstellten Verzeichnis
        with open(file_path, 'wb') as file:
            file.write(response.content)
        print(f'Datei gespeichert: {file_path}')
    else:
        print(f'Fehler beim Abrufen der Datei für ID {item_id}: {response.status_code}')

print('Fertig!')

Datei gespeichert: La_Otra_Alemania\1945-06-15_ZYOWCJLQOY4U2C5DXWWX22WYDP5PQJT3.xml
Datei gespeichert: La_Otra_Alemania\1944-06-01_MMZANT7CHYIXHM2E63NLTPIQL5VWGIFX.xml
Datei gespeichert: La_Otra_Alemania\1947-04-15_GCADKIMUKSSL23LL6BXHWRNP66KQR37R.xml
Datei gespeichert: La_Otra_Alemania\1944-04-25_S3LEVONENQZ7BAICDWHE7L2LPJ6IJYD2.xml
Datei gespeichert: La_Otra_Alemania\1943-03-01_QC2SMVMRK67PTHDQW65FLNUCUOWUO2UV.xml
Datei gespeichert: La_Otra_Alemania\1946-12-01_R447FYT3N4GHLXMZ57UMCK2J33IHLSKD.xml
Datei gespeichert: La_Otra_Alemania\1943-12-01_RESVDPWSQMDVMSCLT6BC2XO5FIO2CWP7.xml
Datei gespeichert: La_Otra_Alemania\1947-12-15_KYFSYF4D2OKSDBQJ2P56UKSTOOT74Q32.xml
Datei gespeichert: La_Otra_Alemania\1947-03-01_2ZEPATYNE5HIR3L2FLJMNA4G7PHJXJWT.xml
Datei gespeichert: La_Otra_Alemania\1948-01-01_QTWE5X2JKYKT2LOIZWKQQ536GUMZX7XT.xml
Datei gespeichert: La_Otra_Alemania\1943-08-01_SYPEUU7HVBAAZS6W3HCUQSJWSY4B5YAZ.xml
Datei gespeichert: La_Otra_Alemania\1944-03-01_UHYDLNVBLM2HINLPULBQRS3LN6KCU

## Download der Bild- und Volltextdaten

Die heruntergeladenen METS/MODS-Daten enthalten Verlinkungen zu allen Bild- und Volltextdaten. Eine stark verkürzte und schematische Darstellung der METS/MODS-Struktur, die insbesondere die Positionen von `mets:fileSec` und `mets:fileGrp` zeigt, sieht wie folgt aus. Hier muss man beachten, dass eine `mets:fileGrp` immer eine zusammengehörige Gruppe von Dateien bildet. Es sind `DEFAULT` (Link zu Bilddateien beim Datenpartner der Deutschen Digitalen Bibliothek) und `DDB_FULLTEXT` (Link zu den Volltextdaten bei der Deutschen Digitalen Bibliothek) interessant.

```XML
<mets:mets xmlns:mets="http://www.loc.gov/METS/" xmlns:mods="http://www.loc.gov/mods/v3">
  
  <!-- Header -->
  <mets:metsHdr CREATEDATE="2024-07-30T12:00:00">
    <!-- Agent Information -->
  </mets:metsHdr>

  <!-- Descriptive Metadata -->
  <mets:dmdSec ID="dmd001">
    <mets:mdWrap MDTYPE="MODS">
      <mets:xmlData>
        <mods:mods>
          <!-- MODS Metadata -->
        </mods:mods>
      </mets:xmlData>
    </mets:mdWrap>
  </mets:dmdSec>

  <!-- File Section -->
  <mets:fileSec>
    <mets:fileGrp USE="DEFAULT">
      <mets:file ID="file0001" MIMETYPE="image/jpeg">
        <mets:FLocat LOCTYPE="URL" xlink:href="http://example.com/image1.jpg"/>
      </mets:file>
    </mets:fileGrp>
    <mets:fileGrp USE="DDB_FULLTEXT">
      <mets:file ID="file0002" MIMETYPE="text/xml">
        <mets:FLocat LOCTYPE="URL" xlink:href="http://example.com/fulltext1.xml"/>
      </mets:file>
    </mets:fileGrp>
  </mets:fileSec>

  <!-- Structural Map -->
  <mets:structMap TYPE="logical">
    <mets:div TYPE="document" DMDID="dmd001">
      <mets:div TYPE="page" ORDER="1">
        <mets:fptr FILEID="file0001"/>
      </mets:div>
      <mets:div TYPE="page" ORDER="2">
        <mets:fptr FILEID="file0002"/>
      </mets:div>
    </mets:div>
  </mets:structMap>

</mets:mets>

```

#### xPath-Querys

Eine Möglichkeit diese Informationen mit einer Abfragesprache für XML-Daten abzufragen ist [xPath](https://de.wikipedia.org/wiki/XPath). Die Abfragen (Querys) sehen wie folgt aus:

- DEFAULT-Bilder: `//mets:mets/mets:fileSec/mets:fileGrp[@USE="DEFAULT"]/mets:file/mets:FLocat/@xlink:href`
- DDB_FULLTEXT-Volltexte: `//mets:mets/mets:fileSec/mets:fileGrp[@USE="DDB_FULLTEXT"]/mets:file/mets:FLocat/@xlink:href`

### KI-generierter Programmcode

<div class="alert alert-block alert-info">
<p><b>Prompt:</b> Erstelle ein möglichst einfaches Python-Skript, das alle METS/MODS-Dateien in dem Verzeichnis <code>La_Otra_Alemania</code> einliest und URLs mittels des XPath-Ausdrucks <code>//mets:mets/mets:fileSec/mets:fileGrp[@USE="DDB_FULLTEXT"]/mets:file/mets:FLocat/@xlink:href</code> extrahiert. Die extrahierten URLs sollen heruntergeladen und in Unterverzeichnissen gespeichert werden. Die Unterverzeichnisse werden nach den Dateinamen der METS/MODS-Dateien benannt. Die heruntergeladenen XML-Dateien sollen mit 1 beginnend durchnummeriert werden.</p>

<p>Das Gleiche soll mit JPEG-Dateien und dem xPath-Ausdruck <code>//mets:mets/mets:fileSec/mets:fileGrp[@USE="DEFAULT"]/mets:file/mets:FLocat/@xlink:href</code> gemacht werden.</p>
</div>

In [34]:
# Python-Bibliothek lxml installieren
%pip install -q lxml

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from lxml import etree

# Definiere die Namensräume
NAMESPACES = {
    'mets': 'http://www.loc.gov/METS/',
    'xlink': 'http://www.w3.org/1999/xlink'
}

# Verzeichnisse definieren
xml_directory = 'La_Otra_Alemania'
download_directory = 'La_Otra_Alemania'

# Erstelle das Download-Verzeichnis, falls es nicht existiert
if not os.path.exists(download_directory):
    os.makedirs(download_directory)

# Funktion, um URLs aus einer XML-Datei zu extrahieren und herunterzuladen
def download_files_from_xml(xml_file_path, xpath_expr, subfolder, extension):
    # Lade die XML-Datei ein
    with open(xml_file_path, 'rb') as xml_file:
        tree = etree.parse(xml_file)

    # Extrahiere die URLs mit dem gegebenen XPath-Ausdruck
    urls = tree.xpath(xpath_expr, namespaces=NAMESPACES)

    # Erstelle das Unterverzeichnis, benannt nach der XML-Datei
    xml_file_name = os.path.splitext(os.path.basename(xml_file_path))[0]
    destination_dir = os.path.join(download_directory, xml_file_name, subfolder)
    os.makedirs(destination_dir, exist_ok=True)

    # Lade jede URL herunter und speichere die Datei
    for i, url in enumerate(urls, start=1):
        try:
            response = requests.get(url)
            response.raise_for_status()  # Überprüfe auf HTTP-Fehler
            file_path = os.path.join(destination_dir, f'{i}.{extension}')
            with open(file_path, 'wb') as output_file:
                output_file.write(response.content)
            print(f'Downloaded {url} to {file_path}')
        except requests.RequestException as e:
            print(f'Failed to download {url}: {e}')

# Durchlaufe alle XML-Dateien im Verzeichnis
for xml_file in os.listdir(xml_directory):
    if xml_file.endswith('.xml'):
        xml_file_path = os.path.join(xml_directory, xml_file)
        # Lade und speichere Dateien mit dem ersten XPath-Ausdruck
        download_files_from_xml(xml_file_path, '//mets:mets/mets:fileSec/mets:fileGrp[@USE="DEFAULT"]/mets:file/mets:FLocat/@xlink:href', 'DEFAULT', 'jpeg')
        # Lade und speichere Dateien mit dem zweiten XPath-Ausdruck
        download_files_from_xml(xml_file_path, '//mets:mets/mets:fileSec/mets:fileGrp[@USE="DDB_FULLTEXT"]/mets:file/mets:FLocat/@xlink:href', 'DDB_FULLTEXT', 'xml')


Downloaded https://portal.dnb.de/bookviewer/view/1026552451/img/page/1/p.jpg?reduce=3 to La_Otra_Alemania\1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4\DEFAULT\1.jpeg
Downloaded https://portal.dnb.de/bookviewer/view/1026552451/img/page/2/p.jpg?reduce=3 to La_Otra_Alemania\1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4\DEFAULT\2.jpeg
Downloaded https://portal.dnb.de/bookviewer/view/1026552451/img/page/3/p.jpg?reduce=3 to La_Otra_Alemania\1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4\DEFAULT\3.jpeg
Downloaded https://portal.dnb.de/bookviewer/view/1026552451/img/page/4/p.jpg?reduce=3 to La_Otra_Alemania\1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4\DEFAULT\4.jpeg
Downloaded https://portal.dnb.de/bookviewer/view/1026552451/img/page/5/p.jpg?reduce=3 to La_Otra_Alemania\1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4\DEFAULT\5.jpeg
Downloaded https://portal.dnb.de/bookviewer/view/1026552451/img/page/6/p.jpg?reduce=3 to La_Otra_Alemania\1942-01-01_BWSVNEKFTM7SPW4SQAWQQNGIHHTRFDS4\DEFAULT\6.jpeg
Downloaded